# Simulacion de una Peluqueria

## Fotografias del sector.
![titulo](p1.png)![titulo](p2.png)

## Recoleccion de datos.
* Fotos Establecimiento.

* Nombre Peluqueria: Peluqueria Estilo Profesional EB
* Tiempo de llegada por cliente a la peluqueria: 15 -45 minutos.
* Tiempo de atencion a niños: 10 - 20 minutos.
* Tiempo de atencion a adultos: 20 - 40 minutos.
* Tiempo de atencion a Mujeres: 30 - 50 minutos.
* Cantidad de sillas de espera: 5
* Cantidad de peluqueros: 1
![titulo](p3.png)
### Objetivo.
La simulación tiene por objetivo estimar la longitud máxima de la fila de clientes; el número de clientes atendidos; y el tiempo de servicio ocioso, durante un día de trabajo, con el fin de identificar oportunidades de mejorar los servicios a los clientes.


In [4]:
import simpy
import random

class Peluqueria:

    def __init__(self, environment, sala_espera, peluqueros):
        self.env = environment
        self.sala_espera = simpy.Resource(self.env, sala_espera)
        self.peluqueros = simpy.Resource(self.env, peluqueros)
        self.totalNinios = 0
        self.totalNinias = 0
        self.totalHombres = 0
        self.totalMujeres = 0
        self.tiempoLlega = 0
        self.tiempoSale = 0
        print('inicio')

    def ingresaSala(self, cliente):
        with self.sala_espera.request() as sala:
            yield sala
            llega = self.env.now
            self.tiempoLlega = llega
            print('Ingresa en sala de espera,', cliente['numero'], cliente['cliente'],' tiempo:', llega)
            yield self.env.process(self.asignarCliente(cliente))

    def asignarCliente(self, cliente):
        i, f = cliente['tiempo_atencion']
        tiempo = random.randint(i, f)
        print('Tiempo de atencion:', tiempo)
        if(tiempo >= (i + f) / 2):
            print('Cobra>', cliente['paga'][1])
        else:
            print('Cobra>', cliente['paga'][0])
        yield self.env.timeout(tiempo)
        with self.peluqueros.request() as corte_peluquero:
            yield corte_peluquero
            yield self.env.process(self.atiendeCliente(cliente))


    def atiendeCliente(self, cliente):
        sale = self.env.now
        self.tiempoSale = sale
        yield self.env.timeout(random.randint(15, 25))
        if(cliente['cliente'] == 'ninios'):
            self.totalNinios += 1
        if(cliente['cliente'] == 'ninias'):
            self.totalNinias += 1
        if(cliente['cliente'] == 'hombres'):
            self.totalHombres += 1
        if(cliente['cliente'] == 'mujeres'):
            self.totalMujeres += 1

    def resultados(self):
        return [('ninios', self.totalNinios), ('ninias', self.totalNinias), ('hombres', self.totalHombres), ('mujeres', self.totalMujeres)]

class Cliente:
    def __init__(self):
        self.clientes = { 'ninios' : [(10, 25) , (1.5, 2)], 'ninias' : [(10, 35), (1.5, 2.5)], 'mujeres' : [(20, 50), (2.5, 7)], 'hombres': [(20, 30), (2.5, 3)],}

    def generaClientes(self, totalClientes):
        self.ninios = totalClientes * .10
        self.ninias = totalClientes * .20
        self.hombre = totalClientes * .25
        self.mujer = totalClientes * .45
        self.clientes['ninios'].append(self.ninios)
        self.clientes['ninias'].append(self.ninias)
        self.clientes['hombres'].append(self.hombre)
        self.clientes['mujeres'].append(self.mujer)

        return self.clientes

import simpy
import random
import matplotlib.pyplot as pp
from Cliente import Cliente
from Peluqueria import Peluqueria

SALA_ESPERA = 6
PELUQUEROS = 2
TOTAL_CLIENTES = 50

def main(env):
    cliente = Cliente()
    clientes = cliente.generaClientes(TOTAL_CLIENTES)
    peluqueria = Peluqueria(env, SALA_ESPERA, PELUQUEROS)
    intervalo = 15
    for cliente in clientes:
        total = clientes[cliente]
        for i in range(int(total[2])):
            yield env.timeout(random.randint(intervalo - 5, intervalo + 5))
            c = {'cliente' : cliente, 'numero' : i, 'paga' : total[1], 'tiempo_atencion' : total[0]}
            env.process(peluqueria.ingresaSala(c))
    datos = sorted(peluqueria.resultados())
    x, y = zip(*datos)
    print(x,y)
    pp.plot(x, y, linewidth = 2, color = 'green')
    pp.scatter(x, y, color='red')
    pp.grid(True)
    pp.title("Clientes Atendidos")
    pp.show()

if __name__ == '__main__':
    random.seed(21)
    env = simpy.Environment()
    env.process(main(env))
    env.run(until = 30000000)


ImportError: No module named 'Cliente'

In [1]:
import random
import math
import simpy

SEMILLA = 30 #Puede ser cualquiera
NUM_PELUQUEROS = 1
TIEMPO_CORTE_MIN = 15 #El tiempo de corte tarde entre 15-30 minutos
TIEMPO_CORTE_MAX = 30
T_LLEGADAS = 20 # Llegan 3 cliente por hora
TIEMPO_SIMULACION = 120 # 120 minutos, sin usar
TOT_CLIENTES = 50

te = 0.0 #Tiempo de espera total.
dt = 0.0 #duracion de servicio total
fin = 0.0 # minuto en el que finaliza.

def cortar(cliente):
    global dt # Acceso a la variable creada anteriormente.
    R = random.random() #Genera numeros aleatorios
    tiempo = TIEMPO_CORTE_MAX - TIEMPO_CORTE_MIN
    tiempo_corte = TIEMPO_CORTE_MIN + (tiempo*R) #Distribucion metodologia uniforme
    yield env.timeout(tiempo_corte) # deja de correr el tiempo n minutos
    print("Corte listo al cliente ",cliente, "y el corte se demoro ", tiempo_corte)
    dt = dt + tiempo_corte #acumula para obtener el total del servicio que dura la simulacion
    
def cliente(env, name, personal):
    global te
    global fin
    llega = env.now #Minuto que llega el cliente
    print(name, "llego a la peluqueria en el minuto: ", llega)
    with personal.request() as request:
        yield request #Obtiene turno
        pasa = env.now #Guadar el minuto cuando comienza a ser atendido
        espera = pasa - llega #Calcula el tiempo que espero.
        esperaHora = espera/60 # lo transforma en horas
        te = te + espera #Acumula los tiempo de espera
        print(name, 'es atendido por el peluquero en el minuto ', pasa, ' y espero para ser atendido en el minuto', espera, esperaHora)
        yield env.process(cortar(name)) #Empieza a cortar
        deja = env.now #Minuto en el que termina el corte de pelo
        print(name, " deja la peluqueria en el minuto ", deja)
        fin = deja #Conserva el ultimo minuto de la simulacion
        
def principal(env, personal):
    llegada = 0
    i = 0
    for i in range(TOT_CLIENTES):
        R = random.random()
        llegada = -T_LLEGADAS * math.log(R) #Distribucion metodologia exponencial.
        yield env.timeout(llegada) #Deja transcurrir un tiempo entre uno y otro
        i += 1
        env.process(cliente(env, 'Cliente %d' %i, personal))
        
print ("------------------- Bienvenido Simulacion Peluqueria ------------------")
random.seed (SEMILLA)  # Cualquier valor
env = simpy.Environment() # Crea el objeto entorno de simulacion
personal = simpy.Resource(env, NUM_PELUQUEROS) #Crea los recursos (peluqueros)
env.process(principal(env, personal)) #Invoca el proceso principal
env.run() #Inicia la simulacion

print ("\n---------------------------------------------------------------------")
print ("\nIndicadores obtenidos: ")

lpc = te / fin
print ("\nLongitud promedio de la cola: %.2f" % lpc)
tep = te / TOT_CLIENTES
print ("Tiempo de espera promedio = %.2f" % tep)
upi = (dt / fin) / NUM_PELUQUEROS
print ("Uso promedio de la instalacion = %.2f" % upi)
print ("\n---------------------------------------------------------------------")

------------------- Bienvenido Simulacion Peluqueria ------------------
Cliente 1 llego a la peluqueria en el minuto:  12.357767874449124
Cliente 1 es atendido por el peluquero en el minuto  12.357767874449124  y espero para ser atendido en el minuto 0.0 0.0
Corte listo al cliente  Cliente 1 y el corte se demoro  15.450553628266906
Cliente 1  deja la peluqueria en el minuto  27.80832150271603
Cliente 2 llego a la peluqueria en el minuto:  37.17074959460619
Cliente 2 es atendido por el peluquero en el minuto  37.17074959460619  y espero para ser atendido en el minuto 0.0 0.0
Cliente 3 llego a la peluqueria en el minuto:  45.674850272890154
Corte listo al cliente  Cliente 2 y el corte se demoro  18.150130433245966
Cliente 2  deja la peluqueria en el minuto  55.320880027852155
Cliente 3 es atendido por el peluquero en el minuto  55.320880027852155  y espero para ser atendido en el minuto 9.646029754962 0.16076716258270002
Cliente 4 llego a la peluqueria en el minuto:  72.8268912880516
Cor